# CV Assignment 3

## Roll no - 20171002

### Requirements :-
##### Libraries:-
* Numpy
* os
* Python 3.x
* scikit-image
* matplotlib
* math
* mpl_toolkits.axes_grid1
* OpenCV 3.4.2.16
* pdb
* PyMaxFlow
* scipy

### To run this notebook successfully, please ensure the following steps.
* Ensure that all the libraries mentioned above are installed
* Ensure that in the current working directory the folder **data** and its entire folder structure exists and is maintained. **This is the input data to notebook.**



### Note:-
* ***If any of the steps are missing/files are missing, then some parts of the code may or may not work***
* *The Results folder contains some outputs saved from the script*
* It is advised to use the testing machine in "plugged in" mode, to avoid core suppression as happens in most modern PCs.
* **Results are uploaded on Microsoft OneDrive at this link, `https://iiitaphyd-my.sharepoint.com/:f:/g/personal/soumyasis_gun_research_iiit_ac_in/Eg_kuIHNtWZLsKbZrYI0WusBblrJ7RBBvFO7MbPoUVx3Hg?e=rOCbEl`**. Please note that this is hosted on IIITH's Microsoft OneDrive.
* ***To run this notebook, download `data` at notebook location and run the notebook***

In [ ]:
import matplotlib.pyplot as plt
import maxflow
import numpy as np
from os import listdir
import pdb
from cv2 import imwrite
import random
from imageio import imread, imsave
from scipy.spatial.distance import cdist
import time
import os

In [ ]:
ip = os.path.join("./","inputs/")
op = os.path.join("./","output/")
if not os.path.exists(ip):
    os.mkdir(ip)
if not os.path.exists(op):
    os.mkdir(op)

SHIFT_DIRECTIONS = ['UP', 'DOWN', 'LEFT', 'RIGHT', 'UP_LEFT', 'UP_RIGHT']
components = 7
boxes_dir = './data/bboxes'
image_dir = './data/images'
files_boxes = []
files_images = []
STRUCTURES = {
    'UP': np.array([[0, 0, 0], [0, 0, 0], [0, 1, 0]]),
    'DOWN': np.array([[0, 1, 0], [0, 0, 0], [0, 0, 0]]),
    'LEFT': np.array([[0, 0, 0], [0, 0, 1], [0, 0, 0]]),
    'RIGHT': np.array([[0, 0, 0], [1, 0, 0], [0, 0, 0]]),
    'UP_LEFT': np.array([[0, 0, 0], [0, 0, 0], [0, 0, 1]]),
    'UP_RIGHT': np.array([[0, 0, 0], [0, 0, 0], [1, 0, 0]]),
}

## Inputs

In [ ]:
for filename in os.listdir(boxes_dir):
    if filename.endswith('.txt'):
        files_boxes.append(boxes_dir + '/' + filename)
        name = filename.split('.')[0]
        files_images.append(image_dir + '/' + name + '.jpg')
print(files_boxes)
print(files_images)

## GrabCit Algorithm

* GrabCut is an image segmentation method based on graph cuts.
* Input is a user-specified bounding box around the object to be segmented.
* The algorithm estimates the color distribution of the target object and that of the background using a Gaussian mixture model.
* This is used to construct a Markov random field over the pixel labels, with an energy function that prefers connected regions having the same label, and running a graph cut based optimization to infer their values. As this estimate is likely to be more accurate than the original, taken from the bounding box, this two-step procedure is repeated until convergence.

### Graph Construction 
The graph has 2 type of edges: 
* **Unary Potentials**: These represent the likelihood of a pixel belonging to either the foreground or background. 
* **Pairwise Potentials**: These enforce the smoothness constraints. These edges are put between all pairs of adjacent pixels.  

## Implementation

In [ ]:
def test_grabcut(thresh, itrs, save_ip = False, stop_by_img = 0, save_op = False):
    cnt = 1
    for box, img_file in zip(files_boxes, files_images):
        contents = open(box)
        line1 = contents.readline().split()
        bounding_box = []
        bounding_box.append(int(line1[0]))
        bounding_box.append(int(line1[1]))
        bounding_box.append(int(line1[2]))
        bounding_box.append(int(line1[3]))
        print('Input bounding box = ')
        print(bounding_box)
        img = imread(img_file)
        plt.figure(figsize = [13,13], dpi = 60)
        width = float(img.shape[1])
        x_range_min = bounding_box[0] / width
        x_range_max = bounding_box[2] / width
#         plt.title('Select Bounding Box',size = 30)
        plt.axhspan(bounding_box[1], bounding_box[3], x_range_min, x_range_max, color='r', fill=False)
        plt.imshow(img)
        plt.title('Input image',size = 30)
        plt.axis('off')
        if save_ip is True:
            plt.savefig('./inputs/'+img_file.split('/')[3][:-4]+'bounded.jpg')
        plt.show()
        start = time.time()
        seg = grabcut(img, bounding_box, thresh=thresh, itrs=itrs)
        end = time.time()
        print('SEGMENTATION TOOK {} SECONDS FOR {} x {} IMAGE'.format(end - start, img.shape[1], img.shape[0]))
        out = np.zeros(img.shape)
        for i in range(img.shape[0]):
            for j in range(img.shape[1]):
                if seg[i,j] == 1:
                    out[i,j,:] = img[i,j,:]
                else:
                    out[i,j,:] = (255,255,255)
        
        plt.figure(figsize = [13,13], dpi = 60)
        plt.imshow(out.astype(np.uint8))
        plt.title('Output \n with Threshold = '+str(thresh)+'\n with '+str(itrs)+' Iterations',size = 30)
        plt.axis('off')
        plt.show()
        if save_op is False:
            print('Saving output at ./output/'+img_file.split('/')[3][:-4]+'_thr_'+str(thresh)+'_itr_'+str(itrs)+'.jpg')
            plt.imsave('./output/'+img_file.split('/')[3][:-4]+'_thr_'+str(thresh)+'_itr_'+str(itrs)+'.jpg', out.astype(np.uint8))
        cnt+=1
        if (stop_by_img) != 0:
            if cnt >= stop_by_img:
                break

In [ ]:
def get_accuracy(seg_map, ground_truth):
    correct = (seg_map + ground_truth - 1) ** 2
    return np.mean(correct)

def grabcut(img, bounding_box=None, thresh = 0.001, itrs = 5):
    if bounding_box is None:
        bounding_box = select_bounding_box(img)
    # print 'BOUNDING BOX COORDS: {}'.format(box)
    # Initialize segmentation map to bounding box, foreground = 1, background = 0
    segmentation_map = np.zeros((img.shape[0], img.shape[1]))
    segmentation_map[bounding_box[1]-1:bounding_box[3] + 1, bounding_box[0]-1:bounding_box[2]+1] = 1
    
    foreground = img[segmentation_map == 1]
    background = img[segmentation_map == 0]

    # Initialize GMM components with k-means
    foreground_ass, background_ass = kmean(foreground, background)

    # Iteratively refine segmentation from initialization
    change = 1
    i = 0
    oldshape = foreground.shape[0]
    itr = 0
    while change > thresh and itr<itrs:
        foreground_gmm, background_gmm = fit_gmm(foreground, background, foreground_ass, background_ass)
        segmentation_map, foreground_ass, background_ass = estimate_segmentation(img, foreground_gmm, background_gmm, segmentation_map, bounding_box)

        foreground = img[segmentation_map == 1]
        background = img[segmentation_map == 0]
        
        change = abs(oldshape-foreground.shape[0])/float(oldshape)
        oldshape = foreground.shape[0]
        print('AT ITERATION {}, {} FOREGROUND / {} BACKGROUND'.format(i + 1, foreground.shape[0], background.shape[0]))
        print('FOREGROUND ASSIGNMENT CHANGED {}%'.format(str(100.0 * change)[:5]))
        i += 1
        itr+=1
    return segmentation_map


In [ ]:
def estimate_segmentation(img, foreground_gmm, background_gmm, seg_map, bounding_box):
    # Calculate unary values and component assignments
    foreground_unary, foreground_ass = get_unary(img, foreground_gmm)
    background_unary, background_ass = get_unary(img, background_gmm)


    pair_pot = get_pairwise(img)

    foreground_unary[:bounding_box[1], :] = 1e4
    background_unary[:bounding_box[1], :] = 0

    foreground_unary[:, :bounding_box[0]] = 1e4
    background_unary[:, :bounding_box[0]] = 0

    foreground_unary[bounding_box[3]:, :] = 1e4
    background_unary[bounding_box[3]:, :] = 0

    foreground_unary[:, bounding_box[2]:] = 1e4
    background_unary[:, bounding_box[2]:] = 0
    pot_graph, nodes = create_graph(foreground_unary, background_unary, pair_pot)
    pot_graph.maxflow()

    # Create bit map of result and return it
    box_seg = segment(pot_graph, nodes)

    seg_map = np.zeros((img.shape[0], img.shape[1]), dtype='int32')
    seg_map[bounding_box[1]-1:bounding_box[3]+1, bounding_box[0]-1:bounding_box[2]+1] = box_seg[bounding_box[1]-1:bounding_box[3]+1, bounding_box[0]-1:bounding_box[2]+1]
    
    return box_seg, foreground_ass[box_seg == 1], background_ass[box_seg == 0]

def create_graph(foreground_unary, background_unary, pair_pot):
    graph = maxflow.Graph[float]()
    nodes = graph.add_grid_nodes(foreground_unary.shape)

    graph.add_grid_tedges(nodes, foreground_unary, background_unary)

    for i, direction in enumerate(SHIFT_DIRECTIONS):
        if i in [0, 2, 4, 5]:
            graph.add_grid_edges(nodes, weights=pair_pot[i], structure=STRUCTURES[direction], symmetric=False)

    return graph, nodes

In [ ]:
def segment(graph, nodes):
    segments = graph.get_grid_segments(nodes)
    return (np.int_(np.logical_not(segments)) - 1) * -1

def get_pairwise(img):
    shifted_imgs = shift(np.array(img))
    pairwise_dist = np.zeros((6, img.shape[0], img.shape[1]))
    i = 0
    while i < 6:
        pairwise_dist[i] = np.sqrt(np.sum((img - shifted_imgs[i]) ** 2, axis=2))
        i+=1

    return np.exp(-1 * (1.0 / (2 * np.mean(pairwise_dist[[0,2,4,5],:,:]))) * pairwise_dist)*50

def shift(img):
    up = img
    down = img
    left = img
    right = img
    up_left = img
    up_right = img
    shifted_imgs = np.zeros((6, up.shape[0], up.shape[1], up.shape[2]), dtype='uint32')
    up[:img.shape[0]-1, : ,:] = img[1:, :, :]
    shifted_imgs[0] = up
    down[1:, :, :] = img[:up.shape[0]-1, :, :]
    shifted_imgs[1] = down
    left[:, :up.shape[1]-1, :] = img[:, 1:, :]
    shifted_imgs[2] = left
    right[:, 1:, :] = img[:, :up.shape[1]-1, :]
    shifted_imgs[3] = right
    up_left[:up.shape[0]-1, :up.shape[1]-1, :] = img[1:, 1:, :]
    shifted_imgs[4] = up_left
    up_right[:up.shape[0]-1, 1:, :] = img[1:, :up.shape[1]-1, :]
    shifted_imgs[5] = up_right
    return shifted_imgs

def get_unary(img, gmm):
    potentials = np.zeros((len(gmm), img.shape[0], img.shape[1]))
    log_pdfs = np.zeros((len(gmm), img.shape[0], img.shape[1]), dtype='float64')
    unary = np.zeros((img.shape[0],img.shape[1]))
    for k, gaussian in enumerate(gmm):
        cov = gaussian['cov']
        piece2 = np.zeros((img.shape[0], img.shape[1]))
        linalg_cov = np.linalg.det(cov)
        log_pdfs[k] +=  -0.5*np.log(linalg_cov)
        reshaped_gaussian = np.reshape(gaussian['mean'], (1, 1, 3))
        piece1 = -np.log(gaussian['size']) + 0.5 * np.log(linalg_cov)
        mu_img = img - reshaped_gaussian
        temp = np.einsum('ijk,il', np.transpose(mu_img), np.linalg.inv(cov))
        i = 0
        j = 0
        while i < img.shape[0]:
            j = 0
            while j < img.shape[1]:
                piece2[i, j] = np.dot(temp[j, i], mu_img[i, j])
                j+=1
            i+=1
        piece2 *= 0.5
        log_pdfs[k] += -1.0 * piece2
        potentials[k] = piece1 + piece2
    assignments = np.argmax(np.array(log_pdfs), axis=0)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            unary[i,j] = potentials[assignments[i,j],i,j]

    return unary, assignments
def findnew_space(inp, inp_mu):
    inp_dist = cdist(inp, inp_mu, metric='euclidean')
    inp_ass = np.argmin(inp_dist, axis=1)
    new_inp_mu = np.zeros_like(inp_mu)
    i = 0
    while i < components:
        new_inp_mu[i] = np.mean(inp[inp_ass == i], axis=0)
        new_inp_mu[i] = np.mean(inp[inp_ass == i], axis=0)
        i = i + 1
    return new_inp_mu, inp_ass


In [ ]:
def kmean(foreground, background):
    avg_centroid_change = float('Inf')
    foreground_mu = foreground[np.random.choice(foreground.shape[0], components), :]
    background_mu = background[np.random.choice(background.shape[0], components), :]
    for j in range(100000):
        if(avg_centroid_change > 1):
            new_foreground_mu, foreground_ass = findnew_space(foreground, foreground_mu)
            new_background_mu, background_ass = findnew_space(background, background_mu)   
            avg_centroid_change = np.mean(2*np.sqrt(np.sum(np.square(foreground_mu - new_foreground_mu), axis=1)))/2
            foreground_mu = new_foreground_mu
            background_mu = new_background_mu
        else:
            break
    return foreground_ass, background_ass

def change(inp_cluster, inp):
    inp_gmm = {}
    inp_gmm['mean'] = np.mean(inp_cluster, axis=0)
    inp_gmm['cov'] = np.cov(inp_cluster, rowvar=0) + np.identity(3)*1e-8
    inp_gmm['size'] = inp_cluster.shape[0] * 1.0 / inp.shape[0]
    return inp_gmm
def findval(inp, inp2, val):
    return inp[inp2 == val]

def fit_gmm(foreground, background, foreground_ass, background_ass, k=components):
    foreground_gmms, background_gmms = [], []
    for i in range(max(np.max(foreground_ass), np.max(background_ass)) + 1):
        foreground_cluster = findval(foreground, foreground_ass, i)
        background_cluster = findval(background, background_ass, i)

        foreground_gmm, background_gmm = {}, {}
        foreground_gmm = change(foreground_cluster, foreground)
        background_gmm = change(background_cluster, background)
        if not foreground_gmm['size'] <= 0.001:
            foreground_gmms.append(foreground_gmm)
        if not background_gmm['size'] <= 0.001:
            background_gmms.append(background_gmm)
    if not len(foreground_gmms) >= k:
        split(foreground_gmms, foreground, foreground_ass, k)

    return foreground_gmms, background_gmms

def split(gmm_list, pixels, assignment, k):
    sizes = np.array([f['size'] for f in gmm_list])
    orig_size = np.max(sizes)
    gmm_list.pop(np.argmax(sizes))

    largest = np.argmax(np.bincount(assignment))
    members = pixels[assignment == largest]

    num_new_comps = k - len(gmm_list)
    ass1, ass2 = kmean(members, members)

    for i in range(num_new_comps):
        new_members = members[ass1 == i]

        new_gmm = {
            'mean': np.mean(new_members, axis=0),
            'cov': np.cov(new_members, rowvar=0) + np.identity(3)*1e-8,
            'size': orig_size * new_members.shape[0] / members.shape[0]
        }

        gmm_list.append(new_gmm)

    return gmm_list

In [ ]:
def select_bounding_box(img, save_ip = False):
    plt.figure(figsize = [13,13], dpi = 60)
#     plt.imshow(img)
#     plt.show()
    click = plt.ginput(2)
#     plt.close()
    bounding_box = []
    bounding_box.append(int(round(min(click[0][0], click[1][0]))))
    bounding_box.append(int(round(min(click[0][1], click[1][1]))))
    bounding_box.append(int(round(max(click[0][0], click[1][0]))))
    bounding_box.append(int(round(max(click[0][1], click[1][1]))))
    
    width = float(img.shape[1])
    x_range_min = bounding_box[0] / width
    x_range_max = bounding_box[2] / width
    plt.title('Select Bounding Box',size = 30)
    plt.axhspan(bounding_box[1], bounding_box[3], x_range_min, x_range_max, color='r', fill=False)
    plt.imshow(img)
    if save_ip is True:
        plt.imsave('./inputs/'+img_file.split('/')[3][:-4]+'bounded.jpg', img.astype(np.uint8))
    plt.show()

## Showing Implementation of GrabCut

Using Threshold `0.1` and `1` iteration with `7` components

In [ ]:
test_grabcut(0.1,1)

## Changing no. of iterations

Using Threshold `0.1` and `3` iterations with 7 components

In [ ]:
test_grabcut(0.1,3,save_ip=True)

## Changing threshold

Using Threshold `0.0001` and `1` iteration with 7 components

*Note - running less number of images because of time expensiveness of algorithm*

In [ ]:
test_grabcut(0.0001,1,save_ip=True,stop_by_img = 5)

## Changing no. of components

Using Threshold `0.1` and `1` iteration with 3 components

*Note - running less number of images because of time expensiveness of algorithm*

In [ ]:
components = 3
test_grabcut(0.1,1,stop_by_img = 2, save_op = False)